# Identifying Ethnicity in OpenSAFELY-TPP
This short report describes how ethnicity can be identified in the OpenSAFELY-TPP database, and the strengths and weaknesses of the methods. Ethnicity is known to be an important determinant of health outcomes, particularly during the COVID-19 outbreak where a complex interplay of social and biological factors resulted in increased exposure, reduced protection, and increased severity of illness. The recording of patients’ ethnic group in primary care can support efforts to achieve equity in service provision and outcomes. This is a living document that will be updated to reflect changes to the OpenSAFELY-TPP database and the patient records within.

## OpenSAFELY
OpenSAFELY is an analytics platform for conducting analyses on Electronic Health Records inside the secure environment where the records are held. This has multiple benefits: 

* We don't transport large volumes of potentially disclosive pseudonymised patient data outside of the secure environments for analysis
* Analyses can run in near real-time as records are ready for analysis as soon as they appear in the secure environment
* All infrastructure and analysis code is stored in GitHub repositories, which are open for security review, scientific review, and re-use

A key feature of OpenSAFELY is the use of study definitions, which are formal specifications of the datasets to be generated from the OpenSAFELY database. This takes care of much of the complex EHR data wrangling required to create a dataset in an analysis-ready format. It also creates a library of standardised and validated variable definitions that can be deployed consistently across multiple projects. 

The purpose of this report is to describe all such variables that relate to BMI, their relative strengths and weaknesses, in what scenarios they are best deployed. It will also describe potential future definitions that have not yet been implemented.

## Available Records
OpenSAFELY-TPP runs inside TPP’s data centre which contains the primary care records for all patients registered at practices using TPP’s SystmOne Clinical Information System. This data centre also imports external datasets from other sources, including A&E attendances and hospital admissions from NHS Digital’s Secondary Use Service, and death registrations from the ONS. More information on available data sources can be found within the [OpenSAFELY documentation](https://docs.opensafely.org/data-sources/intro/). 

#Methods

In OpenSAFELY-TPP, there is no categorical “ethnicity” variable to record this information. Rather, ethnicity is recorded using clinical codes, like any other clinical or administrative event, with specific codes relating to specific ethnic groups

We define three codelists to capture primary care ethnicity in OpenSAFELY-TPP : "[CTV3](https://www.opencodelists.org/codelist/opensafely/ethnicity/2020-04-27)", "[SNOMED](https://www.opencodelists.org/codelist/opensafely/ethnicity-snomed-0removed/2e641f61/)" and "[PRIMIS](https://www.opencodelists.org/codelist/primis-covid19-vacc-uptake/eth2001/v1/)".


To evaluate how well each of these codelists are populated, we count the number of patients with at least one instance each codelist, as well as the grouping of the ethncitity themselves.

We examine trends across the whole population and by each of the following demographic and clinical subgroups to detect any inequalities.

Demographic covariates:

Age band
Sex
Ethnicity
Region
IMD
Clinical covariates:

Dementia
Diabetes
Learning disability

Any counts below 6 were redacted, and all other values were rounded to the nearest 5.

In [1]:
import sys
print(sys.path)

['c:\\Users\\candrews\\Documents\\GitHub\\ethnicity-short-data-report\\notebooks_jupyter', 'c:\\Users\\candrews\\anaconda3\\python39.zip', 'c:\\Users\\candrews\\anaconda3\\DLLs', 'c:\\Users\\candrews\\anaconda3\\lib', 'c:\\Users\\candrews\\anaconda3', '', 'c:\\Users\\candrews\\anaconda3\\lib\\site-packages', 'c:\\Users\\candrews\\anaconda3\\lib\\site-packages\\locket-0.2.1-py3.9.egg', 'c:\\Users\\candrews\\anaconda3\\lib\\site-packages\\win32', 'c:\\Users\\candrews\\anaconda3\\lib\\site-packages\\win32\\lib', 'c:\\Users\\candrews\\anaconda3\\lib\\site-packages\\Pythonwin', 'c:\\Users\\candrews\\anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\candrews\\.ipython']


In [2]:
import os
import pandas as pd
import numpy as np
from itertools import product
from IPython.display import display, Markdown, Image

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.options.mode.chained_assignment = None 
pd.options.display.float_format = '{:,.0f}'.format

In [3]:

def local_patient_counts(
    definitions, output_path, code_dict="", categories=False, missing=False,
):
    import pandas as pd

    suffix = "_filled"
    overlap = "all_filled"
    if missing == True:
        suffix = "_missing"
        overlap = "all_missing"
    if categories:
        df_population = pd.read_csv(
            f"../output/{output_path}/simple_patient_counts_registered.csv"
        ).set_index(["group", "subgroup"])
        

        df_append = pd.read_csv(
            f"../output/{output_path}/simple_patient_counts_categories_registered.csv"
        ).set_index(["group", "subgroup"])
        df_append.drop("population", inplace=True, axis=1)
        df_append["population"] = df_population["ethnicity_new_5_filled"]
        # ensure definitions[n] in code_dict[definitions[n]] below refers to one of the definitions of interest
        definitions = [
            f"{category}_{definition}"
            for category, definition in product(
                code_dict[definitions[1]].values(), definitions
            )
        ]

    else:
        df_append = pd.read_csv(
            f"../output/{output_path}/simple_patient_counts_registered.csv"
        ).set_index(["group", "subgroup"])

    for definition in definitions:
        if missing:
            df_append[definition + suffix] = (
                df_append["population"] - df_append[definition + "_filled"]
            )    
        df_append[definition + "_pct"] = round(
            (df_append[definition + suffix].div(df_append["population"])) * 100, 1
        )
        df_append[overlap + "_pct"] = round(
            (df_append[overlap].div(df_append["population"])) * 100, 1
        )

        # Combine count and percentage columns
        df_append[definition] = (
            df_append[definition + suffix].apply(lambda x: "{:,.0f}".format(x))
            + " ("
            + df_append[definition + "_pct"].astype(str)
            + ")"
        )
        df_append = df_append.drop(columns=[definition + suffix, definition + "_pct"])
    df_append[overlap] = (
        df_append[overlap].apply(lambda x: "{:,.0f}".format(x))
        + " ("
        + df_append[overlap + "_pct"].astype(str)
        + ")"
    )
    df_append = df_append.drop(columns=[overlap + "_pct"])
    df_patient_counts = df_append[definitions + [overlap] + ["population"]]
    # Final redaction step
    df_patient_counts = df_patient_counts.replace(np.nan, "-")
    df_patient_counts = df_patient_counts.replace("nan (nan)", "- (-)")
    df_patient_counts.columns = df_patient_counts.columns.str.replace("_", " ")
    
    display(df_patient_counts)
    if categories:
        df_patient_counts.to_csv(
                f"../output/{output_path}/local_patient_counts_categories_registered.csv"
            )


In [4]:
### CONFIGURE ###
definitions = ['ethnicity_5', 'ethnicity_new_5', 'ethnicity_primis_5']
covariates = ['_age_band','_sex','_region','_imd','_dementia','_diabetes','_hypertension','_learning_disability']
output_path = 'released/output'
suffixes = ['','_missing']

code_dict = {
    "imd": {
        0: "Unknown",
        1: "1 Most deprived",
        2: "2",
        3: "3",
        4: "4",
        5: "5 Least deprived",
    },
    "ethnicity_5": {1: "White", 2: "Mixed", 3: "Asian", 4: "Black", 5: "Other"},
    "ethnicity_new_5": {1: "White", 2: "Mixed", 3: "Asian", 4: "Black", 5: "Other"},
    "ethnicity_primis_5": {1: "White", 2: "Mixed", 3: "Asian", 4: "Black", 5: "Other"},
}



## Results

### Count of Patients

Around 14.8 million patients have all three codelists. 

In [5]:
local_patient_counts(
         definitions,  output_path
    )

ethnicity 5  \
group               subgroup                                      
all                 with records              19,004,920 (76.6)   
age_band            0-19                       3,411,965 (62.6)   
                    20-29                      2,221,440 (72.1)   
                    30-39                      2,919,065 (82.0)   
                    40-49                      2,660,880 (83.3)   
                    50-59                      2,789,055 (82.0)   
                    60-69                      2,237,240 (82.7)   
                    70-79                      1,801,995 (83.0)   
                    80+                          963,275 (77.8)   
                    missing                               - (-)   
sex                 F                          9,695,325 (78.3)   
                    M                          9,309,595 (74.9)   
region              East                       4,378,705 (76.1)   
                    East Midlands              3,301,345 (77.2)   
                    London                     1,445,455 (80.5)   
                    North East                   901,895 (76.8)   
                    North West                 1,651,170 (76.8)   
                    South East                 1,232,315 (75.0)   
                    South West                 2,529,865 (74.3)   
                    West Midlands                819,650 (81.1)   
                    Yorkshire and The Humber   2,715,235 (76.3)   
imd                 1 Most deprived            3,819,585 (77.3)   
                    2                          3,725,010 (77.1)   
                    3                          3,894,385 (77.3)   
                    4                          3,727,520 (76.5)   
                    5 Least deprived           3,322,785 (75.8)   
                    Unknown                      515,635 (70.3)   
dementia            False                     18,969,600 (76.6)   
                    True                          35,320 (80.8)   
diabetes            False                     17,028,265 (75.9)   
                    True                       1,976,655 (83.9)   
hypertension        False                     17,457,130 (76.0)   
                    True                       1,547,790 (83.9)   
learning_disability False                     18,887,970 (76.6)   
                    True                         116,950 (82.9)   

                                                ethnicity new 5  \
group               subgroup                                      
all                 with records              18,968,795 (76.5)   
age_band            0-19                       3,401,310 (62.4)   
                    20-29                      2,215,600 (72.0)   
                    30-39                      2,912,280 (81.8)   
                    40-49                      2,655,330 (83.1)   
                    50-59                      2,785,625 (81.9)   
                    60-69                      2,235,200 (82.6)   
                    70-79                      1,800,755 (82.9)   
                    80+                          962,695 (77.7)   
                    missing                               - (-)   
sex                 F                          9,678,510 (78.2)   
                    M                          9,290,290 (74.8)   
region              East                       4,368,735 (75.9)   
                    East Midlands              3,295,160 (77.0)   
                    London                     1,440,175 (80.3)   
                    North East                   900,925 (76.7)   
                    North West                 1,648,400 (76.7)   
                    South East                 1,229,985 (74.8)   
                    South West                 2,526,855 (74.2)   
                    West Midlands                817,725 (80.9)   
                    Yorkshire and The Humber   2,711,575 (76.2)   
imd                 1 Most depr

### Count of Missings

In [6]:
local_patient_counts(
         definitions,  output_path, missing= True
    )

ethnicity 5  \
group               subgroup                                     
all                 with records              5,797,365 (23.4)   
age_band            0-19                      2,038,195 (37.4)   
                    20-29                       857,780 (27.9)   
                    30-39                       641,850 (18.0)   
                    40-49                       534,125 (16.7)   
                    50-59                       610,770 (18.0)   
                    60-69                       469,280 (17.3)   
                    70-79                       370,060 (17.0)   
                    80+                         275,305 (22.2)   
                    missing                              - (-)   
sex                 F                         2,685,565 (21.7)   
                    M                         3,111,795 (25.1)   
region              East                      1,374,695 (23.9)   
                    East Midlands               976,300 (22.8)   
                    London                      349,115 (19.5)   
                    North East                  272,210 (23.2)   
                    North West                  497,575 (23.2)   
                    South East                  411,670 (25.0)   
                    South West                  873,985 (25.7)   
                    West Midlands               191,415 (18.9)   
                    Yorkshire and The Humber    841,435 (23.7)   
imd                 1 Most deprived           1,123,200 (22.7)   
                    2                         1,108,970 (22.9)   
                    3                         1,143,605 (22.7)   
                    4                         1,142,480 (23.5)   
                    5 Least deprived          1,061,005 (24.2)   
                    Unknown                     218,100 (29.7)   
dementia            False                     5,788,965 (23.4)   
                    True                          8,395 (19.2)   
diabetes            False                     5,418,910 (24.1)   
                    True                        378,455 (16.1)   
hypertension        False                     5,501,220 (24.0)   
                    True                        296,145 (16.1)   
learning_disability False                     5,773,220 (23.4)   
                    True                         24,145 (17.1)   

                                               ethnicity new 5  \
group               subgroup                                     
all                 with records              5,833,490 (23.5)   
age_band            0-19                      2,048,850 (37.6)   
                    20-29                       863,620 (28.0)   
                    30-39                       648,635 (18.2)   
                    40-49                       539,675 (16.9)   
                    50-59                       614,200 (18.1)   
                    60-69                       471,320 (17.4)   
                    70-79                       371,300 (17.1)   
                    80+                         275,885 (22.3)   
                    missing                              - (-)   
sex                 F                         2,702,380 (21.8)   
                    M                         3,131,100 (25.2)   
region              East                      1,384,665 (24.1)   
                    East Midlands               982,485 (23.0)   
                    London                      354,395 (19.7)   
                    North East                  273,180 (23.3)   
                    North West                  500,345 (23.3)   
                    South East                  414,000 (25.2)   
                    South West                  876,995 (25.8)   
                    West Midlands               193,340 (19.1)   
                    Yorkshire and The Humber    845,095 (23.8)   
imd                 1 Most deprived           1,134,405 (23.0)   
                    2   

### Count by Category

In [7]:
local_patient_counts(
         definitions,  output_path,code_dict, categories=True,missing=False
    )

White ethnicity 5  \
group               subgroup                                      
all                 with records              15,879,885 (83.7)   
age_band            0-19                       2,665,795 (78.4)   
                    20-29                      1,707,510 (77.1)   
                    30-39                      2,286,100 (78.5)   
                    40-49                      2,112,890 (79.6)   
                    50-59                      2,442,685 (87.7)   
                    60-69                      2,039,030 (91.2)   
                    70-79                      1,710,380 (95.0)   
                    80+                          915,490 (95.1)   
                    missing                               - (-)   
sex                 F                          8,163,170 (84.3)   
                    M                          7,716,710 (83.1)   
region              East                       3,748,930 (85.8)   
                    East Midlands              2,757,375 (83.7)   
                    London                       762,495 (52.9)   
                    North East                   799,045 (88.7)   
                    North West                 1,527,660 (92.7)   
                    South East                 1,094,080 (89.0)   
                    South West                 2,367,015 (93.7)   
                    West Midlands                528,045 (64.6)   
                    Yorkshire and The Humber   2,271,135 (83.8)   
imd                 1 Most deprived            2,818,890 (74.0)   
                    2                          2,935,460 (79.0)   
                    3                          3,331,130 (85.7)   
                    4                          3,329,980 (89.5)   
                    5 Least deprived           3,039,535 (91.6)   
                    Unknown                      424,890 (82.5)   
dementia            False                     15,846,825 (83.7)   
                    True                          33,060 (93.7)   
diabetes            False                     14,305,260 (84.2)   
                    True                       1,574,625 (79.8)   
hypertension        False                     14,467,615 (83.0)   
                    True                       1,412,270 (91.3)   
learning_disability False                     15,776,945 (83.7)   
                    True                         102,935 (88.1)   

                                             White ethnicity new 5  \
group               subgroup                                         
all                 with records                 15,889,355 (83.8)   
age_band            0-19                          2,666,920 (78.4)   
                    20-29                         1,709,125 (77.1)   
                    30-39                         2,289,385 (78.6)   
                    40-49                         2,115,260 (79.7)   
                    50-59                         2,443,630 (87.7)   
                    60-69                         2,039,290 (91.2)   
                    70-79                         1,710,315 (95.0)   
                    80+                             915,425 (95.1)   
                    missing                                  - (-)   
sex                 F                             8,168,160 (84.4)   
                    M                             7,721,195 (83.1)   
region              East                          3,751,915 (85.9)   
                    East Midlands                 2,758,360 (83.7)   
                    London                          764,615 (53.1)   
                    North East                      799,300 (88.7)   
                    North West                    1,527,455 (92.7)   
                    South East                    1,094,795 (89.0)   
                    South West                    2,368,080 (93.7)   
                    West Midlands                   528,120 (64.6)   
                    Yorksh

### Overlapping Definitions
Idea: Use an upset plot

In [8]:
#display(Image(f"../output/{output_path}/../figures/heatmap.png"))

### Latest vs. Most Common

In [9]:
for definition in definitions:
    for suffix in suffixes:
        df_sum = pd.read_csv(f'../output/{output_path}/simple_latest_common_{definition}{suffix}.csv').set_index(definition)
        # sort rows by category index
        df_sum.columns = df_sum.columns.str.replace(definition + "_", "")
        df_sum.columns = df_sum.columns.str.lower()
        df_sum = df_sum.reindex(list(code_dict[definition].values()))
        
        df_counts = pd.DataFrame(
            np.diagonal(df_sum),
            index=df_sum.index,
        #   columns=[f"matching (n={np.diagonal(df_sum).sum()})"],
        )

        df_sum2 = df_sum.copy(deep=True)
        np.fill_diagonal(df_sum2.values, 0)
        df_diag = pd.DataFrame(
            df_sum2.sum(axis=1),
        )
        df_out = df_counts.merge(df_diag, right_index=True, left_index=True)
        columns=round(df_out.sum()/df_out.sum(axis=1).sum()*100,1)
        df_out.columns=[f"matching ({columns[0]}%)",f"not matching ({columns[1]}%)"]
        display(df_out)
        
        if code_dict != "":
            lowerlist = [x.lower() for x in (list(code_dict[definition].values()))]
            df_sum = df_sum[lowerlist]
        else:
            df_sum = df_sum.reindex(sorted(df_sum.columns), axis=1)

        # Combine count and percentage columns
        df_sum["population"]=df_sum.sum(axis = 1)
        for item in lowerlist:
            df_sum[item + "_pct"]= round(
                    (df_sum[item].div(df_sum["population"])) * 100, 1
                )
        
            df_sum[item] = (
                    df_sum[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_sum[item + "_pct"].astype(str)
                    + ")"
                )
        df_sum = df_sum[lowerlist]

        display(df_sum)
    # df_expanded = pd.read_csv(f'../output/{output_path}/tables/latest_common_expanded_{definition}.csv').set_index(definition)
    
    # display(df_expanded)

,matching (97.7%),not matching (2.3%)
ethnicity_5,,
White,"15,715,175","120,825"
Mixed,"321,470","97,355"
Asian,"1,624,315","67,770"
Black,"555,610","50,225"
Other,"512,885","113,930"


,white,mixed,asian,black,other
ethnicity_5,,,,,
White,"15,715,175 (99.2)","33,505 (0.2)","22,835 (0.1)","16,475 (0.1)","48,010 (0.3)"
Mixed,"44,600 (10.6)","321,470 (76.8)","13,805 (3.3)","25,375 (6.1)","13,575 (3.2)"
Asian,"20,355 (1.2)","11,710 (0.7)","1,624,315 (96.0)","4,315 (0.3)","31,390 (1.9)"
Black,"18,820 (3.1)","19,625 (3.2)","4,135 (0.7)","555,610 (91.7)","7,645 (1.3)"
Other,"61,085 (9.7)","12,765 (2.0)","31,875 (5.1)","8,205 (1.3)","512,885 (81.8)"


,matching (96.8%),not matching (3.2%)
ethnicity_5,,
White,"970,885","10,660"
Mixed,"16,145","8,425"
Asian,"123,900","5,740"
Black,"38,935","4,555"
Other,"19,730","9,160"


,white,mixed,asian,black,other
ethnicity_5,,,,,
White,"970,885 (98.9)","3,040 (0.3)","2,155 (0.2)","1,400 (0.1)","4,065 (0.4)"
Mixed,"4,090 (16.6)","16,145 (65.7)","1,270 (5.2)","2,185 (8.9)",880 (3.6)
Asian,"2,290 (1.8)",805 (0.6),"123,900 (95.6)",510 (0.4),"2,135 (1.6)"
Black,"2,090 (4.8)","1,310 (3.0)",495 (1.1),"38,935 (89.5)",660 (1.5)
Other,"4,580 (15.9)",910 (3.1),"3,025 (10.5)",645 (2.2),"19,730 (68.3)"


,matching (97.9%),not matching (2.1%)
ethnicity_new_5,,
White,"15,557,865","109,605"
Mixed,"317,765","95,425"
Asian,"1,635,455","60,285"
Black,"550,055","48,125"
Other,"426,920","86,965"


...

,white,mixed,asian,black,other
ethnicity_new_5,,,,,
White,"962,340 (99.1)","2,985 (0.3)","2,185 (0.2)","1,410 (0.1)","2,355 (0.2)"
Mixed,"4,145 (17.1)","16,000 (66.0)","1,300 (5.4)","2,195 (9.1)",610 (2.5)
Asian,"2,340 (1.8)",835 (0.6),"123,845 (96.0)",520 (0.4),"1,515 (1.2)"
Black,"2,085 (4.8)","1,265 (2.9)",500 (1.2),"38,755 (90.0)",440 (1.0)
Other,"4,310 (16.4)",825 (3.1),"2,485 (9.5)",640 (2.4),"17,965 (68.5)"


,matching (98.0%),not matching (2.0%)
ethnicity_primis_5,,
White,"12,098,680","86,070"
Mixed,"299,750","70,385"
Asian,"1,419,295","43,705"
Black,"437,190","36,055"
Other,"425,400","67,605"


,white,mixed,asian,black,other
ethnicity_primis_5,,,,,
White,"12,098,680 (99.3)","25,215 (0.2)","17,645 (0.1)","11,535 (0.1)","31,675 (0.3)"
Mixed,"32,760 (8.9)","299,750 (81.0)","10,765 (2.9)","17,695 (4.8)","9,165 (2.5)"
Asian,"16,245 (1.1)","8,785 (0.6)","1,419,295 (97.0)","2,560 (0.2)","16,115 (1.1)"
Black,"13,630 (2.9)","15,065 (3.2)","2,630 (0.6)","437,190 (92.4)","4,730 (1.0)"
Other,"35,640 (7.2)","8,775 (1.8)","18,040 (3.7)","5,150 (1.0)","425,400 (86.3)"


,matching (97.6%),not matching (2.4%)
ethnicity_primis_5,,
White,"875,420","5,930"
Mixed,"16,705","6,345"
Asian,"121,565","3,320"
Black,"36,050","2,985"
Other,"3,165","6,945"


,white,mixed,asian,black,other
ethnicity_primis_5,,,,,
White,"875,420 (99.3)","2,575 (0.3)","1,900 (0.2)","1,230 (0.1)",225 (0.0)
Mixed,"3,295 (14.3)","16,705 (72.5)","1,045 (4.5)","1,905 (8.3)",100 (0.4)
Asian,"2,020 (1.6)",690 (0.6),"121,565 (97.3)",415 (0.3),195 (0.2)
Black,"1,605 (4.1)",975 (2.5),350 (0.9),"36,050 (92.4)",55 (0.1)
Other,"3,650 (36.1)",680 (6.7),"2,080 (20.6)",535 (5.3),"3,165 (31.3)"


### State Change

In [10]:
for definition in definitions:
    for suffix in suffixes:
        df_state_change = pd.read_csv(f'../output/{output_path}/local_state_change_{definition}{suffix}_registered.csv').set_index(definition)
        df_state_change.columns = df_state_change.columns.str.replace(definition + "_", "")
        #resort rows
        df_state_change = df_state_change.reindex(list(code_dict[definition].values()))
        df_state_change = df_state_change.reset_index()
        
        df_state_change[definition]=df_state_change[definition]+": " +df_state_change["n"].apply(lambda x: "{:,.0f}".format(x))
        df_state_change = df_state_change.set_index(definition)
        for item in lowerlist:
            df_state_change[item + "_pct"]= round(
                    (df_state_change[item].div(df_state_change["n"])) * 100, 1
                )
        
            df_state_change[item] = (
                    df_state_change[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_state_change[item + "_pct"].astype(str)
                    + ")"
                )
        df_state_change=df_state_change[lowerlist]
        display(df_state_change)

FileNotFoundError: [Errno 2] No such file or directory: '../output/released/output/local_state_change_ethnicity_5_registered.csv'